# Laboratoire 1: Introduction au Deep learning

Le but de se laboratoire est de se familiariser avec Pytorch en l'utilisant pour faire du classement sur deux jeux de données connus: MNIST et CIFAR-10.

## Mnist

In [ ]:
import random
import numpy as np
import torch.nn as nn
import torch.optim as optim
import time
import matplotlib.pyplot as plt
%matplotlib inline

from deeplib.datasets import load_mnist, load_cifar10, Dataset
from sklearn.metrics import accuracy_score
from deeplib.net import MnistNet, CifarNet
from deeplib.history import History
from deeplib.visualization import plot_images

Mnist est jeu de données contenant des images de chiffres manuscrits.<br>
Le jeu de donnée est séparé comme suit: 50000 images sont utilisées en entraînement et 10000 en test.<br> 
Afin d'accélérer les calculs, nous commencerons par charger uniquement 1000 images pour l'entraînement.<br>
Nous allons aussi charger 1000 autres images pour créer un ensemble de validation qui nous permettra d'estimer, pendant l'entraînement, les capacités de généralisation de notre modèle.

In [ ]:
mnist = load_mnist()
train_set = Dataset(mnist[:1000])
val_set = Dataset(mnist[1000:2000])
test_set = load_mnist(train=False)

Vous pouvez exécuter cette cellule pour visualiser le jeu de donnée.

In [ ]:
idx = random.sample([x for x in range(len(train_set))], 9)
images = [np.array(train_set[i][0]) for i in idx]
targets = [train_set[i][1] for i in idx]

plot_images(images, targets, gray=True)

## Entraînement

Pour l'entraînement, nous avons besoin d'une fonction validate qui nous permettra d'estimer la performance de notre modèle et d'une fonction train pour faire l'entraînement.

Pour chaque image du jeu de donnée, la fonction validate fait prédire une classe au réseau entraîné et compare le résultat avec la vraie réponse. Elle retourne le pourcentage de réponse correcte.

In [ ]:
def validate(model, dataset, use_gpu=False):
    true =[]
    pred = []
    val_loss = []
    criterion = nn.CrossEntropyLoss()
    for j in range(len(dataset) // batch_size):
        inputs, targets = dataset.get_mini_batch(batch_size, use_gpu)
        output = model(inputs)
        val_loss.append(criterion(output, targets).data[0])
        predictions = output.max(dim=1)[1]
        true.extend(targets.data.cpu().numpy().tolist())
        pred.extend(predictions.data.cpu().numpy().tolist())

    return accuracy_score(true, pred) * 100, sum(val_loss) / len(val_loss)

Pour chaque epoch, la fonction passe au travers de toutes les images du jeu de donnée et met à jour les poids du réseau selon la perte calculée. On sauvegarde aussi quelques informations importantes afin de visualiser ce qui se passe pendant l'entraînement.

In [ ]:
def train(model, train_set, val_set, n_epoch, batch_size, lr, use_gpu=False):
    history = History()
    
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=lr)
    
    for i in range(n_epoch):
        for j in range(len(train_set) // batch_size):
            inputs, targets = train_set.get_mini_batch(batch_size, use_gpu)
            optimizer.zero_grad()
            output = model(inputs)

            loss = criterion(output, targets)
            loss.backward()
            optimizer.step()
        
        train_acc, train_loss = validate(model, train_set, use_gpu)
        val_acc, val_loss = validate(model, val_set, use_gpu)
        history.save(train_acc, val_acc, train_loss, val_loss)
        print('Epoch {} - Train acc: {:.2f} - Val acc: {:.2f} - Train loss: {:.4f} - Val loss: {:.4f}'.format(i, train_acc, val_acc, train_loss, val_loss))
        
    return history

Entraînons un modèle.

In [ ]:
model = MnistNet() # On crée le modèle

# On définit les hyperparamètres
epoch = 20
batch_size = 32
lr = 0.1

# On entraîne le modèle
history = train(model, train_set, val_set, epoch, batch_size, lr)

La fonction suivante permet de visualiser l'entraînement précédent.

Le premier graphique vous montre l'évolution de la précision du modèle sur le jeu de donnée d'entraînement et sur celui de validation. Le deuxième montre la perte sur les deux jeux de données.

In [ ]:
history.display()

Finalement, évaluons les performances du modèle sur le jeu de données de test.

In [ ]:
score, loss = validate(model, test_set)
print(score)

## Différences CPU - GPU

Pour faire exécuter le code sur GPU, il faut déplacer le model, les inputs et les targets sur le GPU.

In [ ]:
model_gpu = MnistNet()
model_gpu.cuda()

Comparons le temps d'exécution sur CPU et sur GPU

Pour ce faire, nous utiliserons MNIST au complet

In [ ]:
train_set = Dataset(mnist[:40000])
val_set = Dataset(mnist[40000:])

In [ ]:
epoch = 5
batch_size = 256
lr = 0.1

Pendant l'entraînement, vérifier l'utilisation du CPU avec la commande htop.

In [ ]:
print('Training on CPU')
model = MnistNet()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=lr)

start_cpu = time.time()
history = train(model, train_set, val_set, epoch, batch_size, lr)
end_cpu = time.time()

cpu_time = end_cpu - start_cpu

Pour vous assurer que le réseau entraîne bien sur GPU, utiliser la commande 

watch -n 1 nvidia-smi 

Observer l'utilisation de la carte et la quantité de mémoire utilisée pendant l'entraînement.

In [ ]:
print('Training on GPU')
model_gpu = MnistNet()
model_gpu.cuda()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model_gpu.parameters(), lr=lr)

start_gpu = time.time()
history_gpu = train(model_gpu, train_set, val_set, epoch, batch_size, lr, use_gpu=True)
end_gpu = time.time()

gpu_time = end_gpu - start_gpu

In [ ]:
print('CPU - Training time: {:.2f}s'.format(cpu_time))
print('GPU - Training time: {:.2f}s'.format(gpu_time))
print('Ratio: {:.2f}x'.format((cpu_time) / (gpu_time)))

## CIFAR 10

CIFAR-10 est un jeu de données comportant des images séparés en 10 classe:<br>
0 - Avion<br>
1 - Voiture<br>
2 - Oiseau<br>
3 - Chat<br>
4 - Chevreuil<br>
5 - Chien<br>
6 - Grenouille<br>
7 - Cheval<br>
8 - Bateau<br>
9 - Camion<br>

Le jeu de données contient 50000 images d'entraînement. Nous en utiliserons 40000 pour l'entraînement et 10000 pour la validation.

In [ ]:
cifar = load_cifar10()
train_set = Dataset(cifar[:40000])
val_set = Dataset(cifar[40000:])
test_set = load_cifar10(train=False)

Visualisons quelques exemples du dataset

In [ ]:
label_names = [
    'airplane',
    'automobile',
    'bird',
    'cat',
    'deer',
    'dog',
    'frog',
    'horse',
    'ship',
    'truck'
]

idx = random.sample([x for x in range(len(train_set))], 9)
images = [np.array(train_set[i][0]) for i in idx]
images = np.asarray(images)
targets = [train_set[i][1] for i in idx]

plot_images(images, label_names, targets)

### Exercices

Utilisez les 3 cellules suivantes pour répondre aux questions.

In [ ]:
epoch = 5
batch_size = 64
lr = 0.1

model= CifarNet()
model.cuda()

history = train(model, train_set, val_set, epoch, batch_size, lr, use_gpu=True)

In [ ]:
history.display()

In [ ]:
score, loss = validate(model, test_set, use_gpu=True)
print(score)

### Effet du nombre d'epochs

Modifier le nombre d'epochs et obverser les performances du réseau.

Que se passe-t-il s'il est trop grand?<br> 
S'il est trop petit?

### Effet de la taille de la batch

Modifier la taille de la batch et observer l'utilisation de la carte graphique.

Sur quoi est-ce que la taille de la batch semble avoir le plus d'impact?<br>
Est-ce qu'elle impacte les performances?<br>

### Effet du taux d'apprentissage (lr)

Encore une fois, observer l'impact du taux d'apprentissage sur l'entraînement.

Que se passe-t-il s'il est trop grand?<br> 
S'il est trop petit?

### Défi

Modifier les hyperparamètres pour améliorer les performances du réseau.<br>
Êtes-vous capable d'obtenir plus de 65% en test?